In [4]:
import os
from dotenv import load_dotenv
from langchain_core.messages import SystemMessage, HumanMessage
from langchain_core.prompts import ChatPromptTemplate
from langchain_deepseek import ChatDeepSeek

load_dotenv()

llm = ChatDeepSeek(model="deepseek-chat", temperature=0, max_tokens=None, timeout=None, max_retries=2)

sys_msg = SystemMessage("You are a helpful assistant that translates English to French. Translate the user sentence")
human_msg = HumanMessage("i love programming")
messages = [sys_msg, human_msg]
ai_msg = llm.invoke(messages)
ai_msg.content

"J'adore la programmation."

In [ ]:
#  chaining
prompt = ChatPromptTemplate(
    [
        ("system", "You are a helpful assistant that can make poetries, and then translate into {output_language}"),
        ("human","{input}")
    ],

)

# LCEL: Langchain Expression Language
chain = prompt | llm
chain.invoke({ "output_language": "Chinese", "input": "moonlight" })

AIMessage(content='**Moonlight**  \n\nSilver whispers in the night,  \nA gentle kiss of pale, soft light.  \nThe world is bathed in quiet glow,  \nWhere shadows dance and rivers flow.  \n\nA lover’s sigh, a dreamer’s tune,  \nBeneath the watchful eye of the moon.  \nIt paints the earth in shades of grace,  \nA fleeting touch, a cool embrace.  \n\nOh, moonlight, pure and ever bright,  \nGuard our dreams through endless night.  \n\n---  \n\n**月光**  \n\n夜的银辉轻轻低语，  \n如一抹温柔的浅吻，淡淡的光缕。  \n世界沐浴在静谧里，  \n影子翩跹，河水轻移。  \n\n似情人的叹息，似梦者的旋律，  \n在月亮凝视下悄然凝聚。  \n它用优雅涂抹大地，  \n如凉沁的拥抱，转瞬却清晰。  \n\n啊，月光，皎洁永恒，  \n请守护长夜，让梦安宁。', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 201, 'prompt_tokens': 22, 'total_tokens': 223, 'completion_tokens_details': None, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}, 'prompt_cache_hit_tokens': 0, 'prompt_cache_miss_tokens': 22}, 'model_name': 'deepseek-chat', 'system_fingerprint': 'fp_8802369eaa_prod0425fp8', 'i

In [13]:
from langchain_core.prompts import MessagesPlaceholder


prompt_template = ChatPromptTemplate.from_messages([
    SystemMessage("你是一个幽默的电视台主持人"),
    MessagesPlaceholder("input"),
])

chain = prompt_template | llm
chain.invoke({ "input": [HumanMessage("你好，主持人")] })



AIMessage(content='（夸张地挥手）哇哦！各位观众朋友们，看看谁来了！这位朋友，你今天准备好和我一起嗨翻全场了吗？', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 28, 'prompt_tokens': 13, 'total_tokens': 41, 'completion_tokens_details': None, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}, 'prompt_cache_hit_tokens': 0, 'prompt_cache_miss_tokens': 13}, 'model_name': 'deepseek-chat', 'system_fingerprint': 'fp_8333852bec_prod0820_fp8_kvcache', 'id': '0ee59b8e-cc10-4dc9-8075-17f34092c7ee', 'service_tier': None, 'finish_reason': 'stop', 'logprobs': None}, id='run--0446f7b6-89ba-4a20-a1ee-8d17f8330379-0', usage_metadata={'input_tokens': 13, 'output_tokens': 28, 'total_tokens': 41, 'input_token_details': {'cache_read': 0}, 'output_token_details': {}})

In [4]:
# streaming output
for token in llm.stream(messages):
    print(token.content, end="|")

|J|'ad|ore| la| program|mation|.||

In [5]:
# document loaders
from langchain_core.documents import Document

documents = [
    Document(
        page_content="Dogs are great companions, known for their loyalty and friendliness.",
        metadata={"source": "mammal-pets-doc"},
    ),
    Document(
        page_content="Cats are independent pets that often enjoy their own space.",
        metadata={"source": "mammal-pets-doc"},
    ),
]

In [6]:
# 将pdf加载到`Document`序列中
from langchain_community.document_loaders import PyPDFLoader

file_path = "docs/nke-10k-2023.pdf"
loader = PyPDFLoader(file_path)

docs = loader.load()

print(len(docs))

107


In [7]:
print(f"{docs[0].page_content[:200]}\n")
print(docs[0].metadata)

Table of Contents
UNITED STATES
SECURITIES AND EXCHANGE COMMISSION
Washington, D.C. 20549
FORM 10-K
(Mark One)
☑  ANNUAL REPORT PURSUANT TO SECTION 13 OR 15(D) OF THE SECURITIES EXCHANGE ACT OF 1934
F

{'producer': 'EDGRpdf Service w/ EO.Pdf 22.0.40.0', 'creator': 'EDGAR Filing HTML Converter', 'creationdate': '2023-07-20T16:22:00-04:00', 'title': '0000320187-23-000039', 'author': 'EDGAR Online, a division of Donnelley Financial Solutions', 'subject': 'Form 10-K filed on 2023-07-20 for the period ending 2023-05-31', 'keywords': '0000320187-23-000039; ; 10-K', 'moddate': '2023-07-20T16:22:08-04:00', 'source': 'docs/nke-10k-2023.pdf', 'total_pages': 107, 'page': 0, 'page_label': '1'}


In [12]:
from langchain.prompts import FewShotPromptTemplate, PromptTemplate

example_prompt = PromptTemplate(
    input_variables=["question", "answer"],
    template="Q: {question}\nA: {answer}",
)
examples = [
    {"question": "如何写好技术文章标题？", "answer": "简洁、具体、可检索，避免空话。"},
    {"question": "什么是Few-shot？", "answer": "在提示中加入少量示例，帮助模型模仿格式与风格。"},
]
prompt = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
    prefix="请模仿示例的风格，用短句给出清晰答案：",
    suffix="问题：{q}\n答案：",
    input_variables=["q"],
)
chain = prompt | llm
res = chain.invoke({ "q": "如何降低大模型幻觉？" })
print(res.content)


提供事实依据，限制自由发挥，加入外部验证。


In [ ]:
# 结构化输出
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field
from typing import List

class Article(BaseModel):
    title: str = Field(description="文章标题")
    tags: List[str] = Field(description="文章标签")
    summary: str = Field(description="文章摘要")

prompt = ChatPromptTemplate.from_messages([
    ("system", "你是一位专业的技术文章作者，请根据下面的主题，生成对应的文章标题、标签和摘要。"),
    ("human", "主题: {topic}")
])

structured_llm = llm.with_structured_output(Article)

chain = prompt | structured_llm
resp = chain.invoke({"topic": "为‘LangChain JSON输出’生成标题、标签和摘要"})
print(resp)

setup='Why did the American and Chinese diplomats always carry thermometers to their meetings?' punchline='Because they needed to check if the temperature was rising or if it was just cold relations!' rating=7


In [ ]:
from langchain_core.output_parsers import JsonOutputParser

# 使用JsonOutputParser保证结果符合json格式
# 只保证是 JSON，可在下游再做 Pydantic 校验
parser = JsonOutputParser()
prompt = ChatPromptTemplate.from_messages([
    ("system", "你是一个信息抽取助手。严格输出 JSON，无解释文本。"),
    ("user", """从以下文本中抽取商品数组，每个包含 name, price(float), tags(list[str])：
文本：{text}
只输出 JSON，对象形如：{{"ok": true, "category":"...", "items":[...]}}""")
])
chain = prompt | llm | parser
resp = chain.invoke({"text": "iPhone 9999；牛奶 6.5；键盘 299"})
print(resp)


{'ok': True, 'category': '商品', 'items': [{'name': 'iPhone', 'price': 9999.0, 'tags': []}, {'name': '牛奶', 'price': 6.5, 'tags': []}, {'name': '键盘', 'price': 299.0, 'tags': []}]}


In [ ]:
from langchain_core.output_parsers import PydanticOutputParser
from langchain.output_parsers.fix import OutputFixingParser
from typing import List, Literal, Optional
from pydantic import BaseModel, Field

class Item(BaseModel):
    name: str = Field(..., description="物品名称")
    price: float = Field(..., ge=0, description="价格，非负")
    tags: List[str] = Field(default_factory=list, description="标签列表")

class Result(BaseModel):
    ok: bool = Field(..., description="处理是否成功")
    category: Literal["food", "tech", "other"] = Field(..., description="分类")
    items: List[Item] = Field(default_factory=list, description="物品数组")
    notes: Optional[str] = Field(None, description="备注，可为空")

base_parser = PydanticOutputParser(pydantic_object=Result)
parser = OutputFixingParser.from_llm(parser=base_parser, llm=llm)

prompt = ChatPromptTemplate.from_messages([
    ("system", "请严格按照给定 JSON Schema 输出，无解释。"),
    ("user", """文本：{text}
格式要求：
{format_instructions}
""")
]).partial(format_instructions=base_parser.get_format_instructions())
chain = prompt | llm | parser
resp = chain.invoke({"text": "小米平板 1999；黄瓜 2.5; 苹果 6.99"})
print(resp)

ok=True category='other' items=[Item(name='小米平板', price=1999.0, tags=[]), Item(name='黄瓜', price=2.5, tags=[]), Item(name='苹果', price=6.99, tags=[])] notes=None
